<h2 align="center">Automatic Machine Learning with H2O AutoML</h2>

### Task 2: Importing Packages

In [1]:
import pandas as pd
pd.options.display.max_rows = 999
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

### Task 3: Loading and Exploring the Data

In [2]:
xl=pd.ExcelFile("bank_term_deposit_marketing_analysis.xlsx")

In [3]:
xl.sheet_names

['PROCEDURE',
 'DATA DESCRIPTION',
 'Step 1 - Collect Information',
 'CLIENT_INFO',
 'LOAN_HISTORY',
 'MARKETING HISTORY',
 'SUBSCRIPTION HISTORY',
 'Step 2 - Merge Information',
 'CLIENT_MERGE',
 'Step 3 - Marketing Analysis',
 'DAILY RANGE',
 'JOB ANALYSIS',
 'Sheet3']

In [4]:
client_info=pd.read_excel(xl,'CLIENT_INFO')
loan_history=pd.read_excel(xl,'LOAN_HISTORY')
marketing_history=pd.read_excel(xl,'MARKETING HISTORY')
subscription_history=pd.read_excel(xl,'SUBSCRIPTION HISTORY')

In [5]:
client_info.head()

,ID,AGE,JOB,MARITAL,EDUCATION
0,2836,58,management,married,tertiary
1,2837,44,technician,single,secondary
2,2838,33,entrepreneur,married,secondary
3,2839,47,blue-collar,married,unknown
4,2840,33,unknown,single,unknown


In [6]:
loan_history.head()

,ID,DEFAULT,BALANCE,HOUSING,LOAN
0,2836,no,2143,yes,no
1,2837,no,29,yes,no
2,2838,no,2,yes,yes
3,2839,no,1506,yes,no
4,2840,no,1,no,no


In [7]:
marketing_history.head()

,ID,CONTACT,DAY,MONTH,DURATION,CAMPAIGN,PDAYS,PREVIOUS,POUTCOME
0,2836,unknown,5,may,261,1,-1,0,unknown
1,2837,unknown,5,may,151,1,-1,0,unknown
2,2838,unknown,5,may,76,1,-1,0,unknown
3,2839,unknown,5,may,92,1,-1,0,unknown
4,2840,unknown,5,may,198,1,-1,0,unknown


In [8]:
subscription_history.head()

,ID,TERM_DEPOSIT
0,2836,no
1,2837,no
2,2838,no
3,2839,no
4,2840,no


In [9]:
df=pd.merge(client_info,loan_history,on=['ID'])
df=pd.merge(df,marketing_history,on=['ID'])
df=pd.merge(df,subscription_history,on=['ID'])
df.head()

,ID,AGE,JOB,MARITAL,EDUCATION,DEFAULT,BALANCE,HOUSING,LOAN,CONTACT,DAY,MONTH,DURATION,CAMPAIGN,PDAYS,PREVIOUS,POUTCOME,TERM_DEPOSIT
0,2836,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,2837,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,2838,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,2839,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,2840,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [10]:
df=df.drop(['ID'],axis=1)

### Task 4: Data Prep & Start H2O

In [11]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Corretto-11.0.22.7.1 (build 11.0.22+7-LTS, mixed mode)
  Starting server from C:\Users\ishim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ishim\AppData\Local\Temp\tmpiuer5_iq
  JVM stdout: C:\Users\ishim\AppData\Local\Temp\tmpiuer5_iq\h2o_ishim_started_from_python.out
  JVM stderr: C:\Users\ishim\AppData\Local\Temp\tmpiuer5_iq\h2o_ishim_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Africa/Kigali
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_ishim_k6ibcl
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.912 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [12]:
h2o_df=h2o.H2OFrame(df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [13]:
h2o_df.describe()

Rows:45211
Cols:17

,AGE,JOB,MARITAL,EDUCATION,DEFAULT,BALANCE,HOUSING,LOAN,CONTACT,DAY,MONTH,DURATION,CAMPAIGN,PDAYS,PREVIOUS,POUTCOME,TERM_DEPOSIT
type,int,enum,enum,enum,enum,int,enum,enum,enum,int,enum,int,int,int,int,enum,enum
mins,18.0,,,,,-8019.0,,,,1.0,,0.0,1.0,-1.0,0.0,,
mean,40.93621021432809,,,,,1362.272057685082,,,,15.806418791886935,,258.1630797814693,2.7638406582468984,40.1978279622216,0.5803233726305563,,
maxs,95.0,,,,,102127.0,,,,31.0,,4918.0,63.0,871.0,275.0,,
sigma,10.61876204097539,,,,,3044.7658291685234,,,,8.32247615304459,,257.5278122651711,3.0980208832791796,100.1287459905981,2.303441044931215,,
zeros,0,,,,,3514,,,,0,,3,0,0,36954,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,58.0,management,married,tertiary,no,2143.0,yes,no,unknown,5.0,may,261.0,1.0,-1.0,0.0,unknown,no
1,44.0,technician,single,secondary,no,29.0,yes,no,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown,no
2,33.0,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown,no


In [14]:
train, test= h2o_df.split_frame(ratios=[.75])
x=train.columns
y='TERM_DEPOSIT'
x.remove(y)

### Task 5: Run H2O AutoML

In [15]:
from h2o.automl import H2OAutoML

In [16]:
df.TERM_DEPOSIT.value_counts()

TERM_DEPOSIT
no     39922
yes     5289
Name: count, dtype: int64

In [17]:
aml=H2OAutoML(max_runtime_secs=600,balance_classes=True,stopping_metric='logloss',project_name='Final',seed=1)
%time aml.train(x=x,y=y,training_frame=train)

AutoML progress: |
13:39:49.706: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
CPU times: total: 7.41 s
Wall time: 10min 3s


key,value
Stacking strategy,cross_validation
Number of base models (used / total),11/39
# GBM base models (used / total),9/26
# DRF base models (used / total),2/2
# GLM base models (used / total),0/1
# DeepLearning base models (used / total),0/10
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


### Task 6: AutoML Leaderboard and Ensemble Exploration

In [18]:
lb=aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_4_AutoML_1_20240402_133949,0.938301,0.191356,0.639063,0.175738,0.245884,0.060459
StackedEnsemble_AllModels_3_AutoML_1_20240402_133949,0.938244,0.19141,0.638769,0.176,0.245921,0.060477
StackedEnsemble_AllModels_2_AutoML_1_20240402_133949,0.937653,0.192097,0.63484,0.170514,0.246372,0.0606991
StackedEnsemble_AllModels_1_AutoML_1_20240402_133949,0.937411,0.192447,0.633839,0.177177,0.246578,0.0608008
StackedEnsemble_BestOfFamily_4_AutoML_1_20240402_133949,0.937398,0.192591,0.635647,0.163439,0.246566,0.060795
GBM_grid_1_AutoML_1_20240402_133949_model_3,0.936758,0.196509,0.634442,0.163507,0.249145,0.062073
StackedEnsemble_BestOfFamily_3_AutoML_1_20240402_133949,0.93659,0.193489,0.628092,0.165426,0.247425,0.0612189
StackedEnsemble_BestOfFamily_2_AutoML_1_20240402_133949,0.936349,0.193889,0.626823,0.175603,0.24757,0.0612908
GBM_grid_1_AutoML_1_20240402_133949_model_6,0.936097,0.196886,0.631394,0.170467,0.249307,0.0621542
StackedEnsemble_BestOfFamily_1_AutoML_1_20240402_133949,0.936062,0.19522,0.625161,0.16725,0.248139,0.0615732


In [19]:
se= aml.leader
metalearner = h2o.get_model(se.metalearner()['name'])

C:\Users\ishim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\h2o\estimators\stackedensemble.py:965: H2ODeprecationWarning: The usage of stacked_ensemble.metalearner()['name'] will be deprecated. Metalearner now returns the metalearner object. If you need to get the 'name' please use stacked_ensemble.metalearner().model_id
  warnings.warn(


In [20]:
metalearner.varimp()

[('GBM_grid_1_AutoML_1_20240402_133949_model_3',
  0.6732324361801147,
  1.0,
  0.24987568022523549),
 ('DRF_1_AutoML_1_20240402_133949',
  0.6449471712112427,
  0.9579858850394061,
  0.23937737467039583),
 ('GBM_1_AutoML_1_20240402_133949',
  0.5222402215003967,
  0.7757205289506848,
  0.19383369483623183),
 ('GBM_2_AutoML_1_20240402_133949',
  0.3406783938407898,
  0.5060338384374066,
  0.12644554959653387),
 ('GBM_grid_1_AutoML_1_20240402_133949_model_6',
  0.20041394233703613,
  0.2976890767090401,
  0.0743852605382937),
 ('GBM_grid_1_AutoML_1_20240402_133949_model_2',
  0.12710461020469666,
  0.1887975138659116,
  0.04717590720207799),
 ('XRT_1_AutoML_1_20240402_133949',
  0.06548649817705154,
  0.09727175141563064,
  0.02430584505168072),
 ('GBM_grid_1_AutoML_1_20240402_133949_model_4',
  0.05439046770334244,
  0.08079002849588038,
  0.020187463325824266),
 ('GBM_grid_1_AutoML_1_20240402_133949_model_1',
  0.035033658146858215,
  0.05203798311566409,
  0.013003026428575884),
 ('G

### Task 7: Base Learner XGBoost Model Exploration

In [21]:
model=h2o.get_model('DRF_1_AutoML_1_20240402_130300')

H2OResponseError: Server error water.exceptions.H2OKeyNotFoundArgumentException:
  Error: Object 'DRF_1_AutoML_1_20240402_130300' not found for argument: key
  Request: GET /3/Models/DRF_1_AutoML_1_20240402_130300


In [ ]:
model.model_performance(test)

In [ ]:
model.varimp_plot(num_of_features=20)